In [1]:
import sys
import os
# Append the library path to PYTHONPATH, so library can be imported.
sys.path.append(os.path.dirname(os.getcwd()))
import datetime
import shutil

import numpy as np
import pandas as pd

from library import network as nw
from library import common as cm

Using TensorFlow backend.


In [2]:
%run setup.py
%load_ext autoreload
%autoreload 2

sns.set(style='darkgrid')
np.random.seed(RANDOM_SEED)

##### Step 0: Set save directory

In [3]:
%run Load_Clean_aux.py

/home/weiguan/Programs/Anaconda3/lib/python3.6/site-packages/numpy/lib/arraysetops.py:463: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


We remove samples when the next trade is not available. 184757 samples ( 5.79%) are removed. We have  94.21% of original data left, which is 3006005.
We remove unreasonable implvol0. 1019 samples ( 0.03%) are removed. We have  94.18% of original data left, which is 3004986.
We remove samples that have time to maturity less than 1 day. 0 samples ( 0.00%) are removed. We have  94.18% of original data left, which is 3004986.
We remove in-the-money samples. 594074 samples ( 19.77%) are removed. We have  75.56% of original data left, which is 2410912.
We shrink moneyness range. 79041 samples ( 3.28%) are removed. We have  73.08% of original data left, which is 2331871.


In [4]:
if FEATURE_SET == 'normal_feature':
    ori_fea = ['M0', 'tau0_implvol0']
    sub_res = res_dir + 'Network/Normal_Feature/TuneHypers/'

if FEATURE_SET == 'delta_vega':
    ori_fea = ['delta_bs', '1_over_sqrt_tau', 'vega_n']
    sub_res = res_dir + 'Network/Delta_Vega/TuneHypers/'
    
if VIX:
    ori_fea += ['fake_vix']

use_fea = [x + '_t' for x in ori_fea] + ['cp_int']
os.makedirs(sub_res, exist_ok=True)
shutil.copy('setup.py', sub_res)

## Tune hyperparameters

In [6]:
hypers = {
    'nodes_per_layer': (30, 30),
    'reg_alpha': 1e-2,
    'lr': 1e-4,
    'epochs': 200,
    'outact': 'linear'
}

##### Step 3: Setup hypers and target
Here, we specify hyperparameters, which will be used for training at the next step.

In [7]:
lab_tune = 'reg_alpha'
value_set = [1e-3, 1e-4, 1e-5, 1e-6]
num_run = NUM_REPEATS
end_period = NUM_PERIOD_END

##### Step 4: Run for each hyper value

In [9]:
for value in value_set:
    # For instance, we tune regularization alpha
    hypers[lab_tune] = value
    alpha_dir = sub_res + f'para={value:.0e}/'
    
    for k in range(num_run):
        print('This is value={} in repeat {}'.format(value, k))
        alpha_dir_dict = {
            'ckp': alpha_dir + f'run{k}/ckp/',
            'history': alpha_dir + f'/run{k}/history/',
            'plot': alpha_dir + f'/run{k}/plot/'
        }
        for key, value in alpha_dir_dict.items():
            os.makedirs(value, exist_ok=True)
        
        df_return = nw.rolling_net(
            df.copy(),
            ori_fea=ori_fea,
            use_fea=use_fea,
            end_period=end_period,
            hypers=hypers,
            sub_res_dir=alpha_dir_dict,
            tune=True
        )  
        

This is value=0.001 in repeat 0


 Working on period 0.

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Use tf.cast instead.
Train on 131068 samples, validate on 34709 samples
Epoch 1/200
131068/131068 [==============================] - 2s 17us/step - loss: 0.2373 - mean_squared_error: 0.2069 - val_loss: 0.0424 - val_mean_squared_error: 0.0150
Epoch 2/200
131068/131068 [==============================] - 2s 15us/step - loss: 0.0603 - mean_squared_error: 0.0350 - val_loss: 0.0360 - val_mean_squared_error: 0.0125
Epoch 3/200
131068/131068 [==============================] - 2s 16us/step - loss: 0.0532 - mean_squared_error: 0.0313 - val_loss: 0.0321 - val_mean_squared_error: 0.0117
Epoch 4/200
131068/131068 [==============================] - 2s 15us/step - loss: 0.0493 - mean_squared_error: 0.0302 - val_loss: 0.0293 - val_mean_squared_error: 0.0114
Epoch 5/200
131068/131068 [==============================] - 2s 16us/step - loss: 0.0466 - m

Epoch 45/200
131068/131068 [==============================] - 2s 13us/step - loss: 0.0313 - mean_squared_error: 0.0284 - val_loss: 0.0131 - val_mean_squared_error: 0.0102
Epoch 46/200
131068/131068 [==============================] - 2s 13us/step - loss: 0.0312 - mean_squared_error: 0.0284 - val_loss: 0.0128 - val_mean_squared_error: 0.0101
Epoch 47/200
131068/131068 [==============================] - 2s 13us/step - loss: 0.0312 - mean_squared_error: 0.0284 - val_loss: 0.0128 - val_mean_squared_error: 0.0101
Epoch 48/200
131068/131068 [==============================] - 2s 14us/step - loss: 0.0311 - mean_squared_error: 0.0284 - val_loss: 0.0126 - val_mean_squared_error: 0.0099
Epoch 49/200
131068/131068 [==============================] - 2s 14us/step - loss: 0.0311 - mean_squared_error: 0.0284 - val_loss: 0.0130 - val_mean_squared_error: 0.0104
Epoch 50/200
131068/131068 [==============================] - 2s 14us/step - loss: 0.0310 - mean_squared_error: 0.0284 - val_loss: 0.0130 - val_m

Epoch 93/200
131068/131068 [==============================] - 2s 13us/step - loss: 0.0300 - mean_squared_error: 0.0283 - val_loss: 0.0117 - val_mean_squared_error: 0.0100
Epoch 94/200
131068/131068 [==============================] - 2s 14us/step - loss: 0.0300 - mean_squared_error: 0.0282 - val_loss: 0.0119 - val_mean_squared_error: 0.0102
Epoch 95/200
131068/131068 [==============================] - 2s 14us/step - loss: 0.0300 - mean_squared_error: 0.0283 - val_loss: 0.0119 - val_mean_squared_error: 0.0102
Epoch 96/200
131068/131068 [==============================] - 2s 14us/step - loss: 0.0299 - mean_squared_error: 0.0282 - val_loss: 0.0115 - val_mean_squared_error: 0.0099
Epoch 97/200
131068/131068 [==============================] - 2s 14us/step - loss: 0.0299 - mean_squared_error: 0.0282 - val_loss: 0.0115 - val_mean_squared_error: 0.0098
Epoch 98/200
131068/131068 [==============================] - 2s 13us/step - loss: 0.0300 - mean_squared_error: 0.0283 - val_loss: 0.0116 - val_m

Epoch 141/200
131068/131068 [==============================] - 2s 14us/step - loss: 0.0296 - mean_squared_error: 0.0282 - val_loss: 0.0123 - val_mean_squared_error: 0.0109
Epoch 142/200
131068/131068 [==============================] - 2s 14us/step - loss: 0.0296 - mean_squared_error: 0.0282 - val_loss: 0.0112 - val_mean_squared_error: 0.0098
Epoch 143/200
131068/131068 [==============================] - 2s 14us/step - loss: 0.0296 - mean_squared_error: 0.0282 - val_loss: 0.0113 - val_mean_squared_error: 0.0099
Epoch 144/200
131068/131068 [==============================] - 2s 14us/step - loss: 0.0296 - mean_squared_error: 0.0282 - val_loss: 0.0112 - val_mean_squared_error: 0.0099
Epoch 145/200
131068/131068 [==============================] - 2s 14us/step - loss: 0.0295 - mean_squared_error: 0.0282 - val_loss: 0.0114 - val_mean_squared_error: 0.0100
Epoch 146/200
131068/131068 [==============================] - 2s 14us/step - loss: 0.0295 - mean_squared_error: 0.0282 - val_loss: 0.0116 -

135130/135130 [==============================] - 2s 15us/step - loss: 0.0257 - mean_squared_error: 0.0222 - val_loss: 0.0125 - val_mean_squared_error: 0.0090
Epoch 36/200
135130/135130 [==============================] - 2s 15us/step - loss: 0.0256 - mean_squared_error: 0.0222 - val_loss: 0.0130 - val_mean_squared_error: 0.0097
Epoch 37/200
135130/135130 [==============================] - 2s 15us/step - loss: 0.0255 - mean_squared_error: 0.0222 - val_loss: 0.0125 - val_mean_squared_error: 0.0093
Epoch 38/200
135130/135130 [==============================] - 2s 15us/step - loss: 0.0254 - mean_squared_error: 0.0222 - val_loss: 0.0131 - val_mean_squared_error: 0.0099
Epoch 39/200
135130/135130 [==============================] - 2s 15us/step - loss: 0.0253 - mean_squared_error: 0.0222 - val_loss: 0.0127 - val_mean_squared_error: 0.0096
Epoch 40/200
135130/135130 [==============================] - 2s 15us/step - loss: 0.0252 - mean_squared_error: 0.0221 - val_loss: 0.0122 - val_mean_squared_e

...

Epoch 79/200
176599/176599 [==============================] - 2s 12us/step - loss: 0.0168 - mean_squared_error: 0.0167 - val_loss: 0.0058 - val_mean_squared_error: 0.0058
Epoch 80/200
176599/176599 [==============================] - 2s 12us/step - loss: 0.0168 - mean_squared_error: 0.0167 - val_loss: 0.0058 - val_mean_squared_error: 0.0057
Epoch 81/200
176599/176599 [==============================] - 2s 12us/step - loss: 0.0167 - mean_squared_error: 0.0167 - val_loss: 0.0058 - val_mean_squared_error: 0.0057
Epoch 82/200
176599/176599 [==============================] - 2s 12us/step - loss: 0.0168 - mean_squared_error: 0.0167 - val_loss: 0.0057 - val_mean_squared_error: 0.0057
Epoch 83/200
176599/176599 [==============================] - 2s 12us/step - loss: 0.0168 - mean_squared_error: 0.0167 - val_loss: 0.0058 - val_mean_squared_error: 0.0058
Epoch 84/200
176599/176599 [==============================] - 2s 12us/step - loss: 0.0167 - mean_squared_error: 0.0167 - val_loss: 0.0058 - val_m

Epoch 127/200
176599/176599 [==============================] - 2s 12us/step - loss: 0.0167 - mean_squared_error: 0.0166 - val_loss: 0.0057 - val_mean_squared_error: 0.0057
Epoch 128/200
176599/176599 [==============================] - 2s 12us/step - loss: 0.0167 - mean_squared_error: 0.0166 - val_loss: 0.0057 - val_mean_squared_error: 0.0056
Epoch 129/200
176599/176599 [==============================] - 2s 13us/step - loss: 0.0167 - mean_squared_error: 0.0167 - val_loss: 0.0058 - val_mean_squared_error: 0.0057
Epoch 130/200
176599/176599 [==============================] - 2s 12us/step - loss: 0.0167 - mean_squared_error: 0.0167 - val_loss: 0.0058 - val_mean_squared_error: 0.0057
Epoch 131/200
176599/176599 [==============================] - 2s 12us/step - loss: 0.0167 - mean_squared_error: 0.0167 - val_loss: 0.0058 - val_mean_squared_error: 0.0057
Epoch 132/200
176599/176599 [==============================] - 2s 12us/step - loss: 0.0167 - mean_squared_error: 0.0167 - val_loss: 0.0058 -

200484/200484 [==============================] - 2s 12us/step - loss: 0.0069 - mean_squared_error: 0.0068 - val_loss: 0.0056 - val_mean_squared_error: 0.0055
Epoch 22/200
200484/200484 [==============================] - 2s 12us/step - loss: 0.0069 - mean_squared_error: 0.0068 - val_loss: 0.0055 - val_mean_squared_error: 0.0055
Epoch 23/200
200484/200484 [==============================] - 2s 12us/step - loss: 0.0069 - mean_squared_error: 0.0068 - val_loss: 0.0054 - val_mean_squared_error: 0.0054
Epoch 24/200
200484/200484 [==============================] - 2s 12us/step - loss: 0.0069 - mean_squared_error: 0.0068 - val_loss: 0.0054 - val_mean_squared_error: 0.0054
Epoch 25/200
200484/200484 [==============================] - 2s 12us/step - loss: 0.0069 - mean_squared_error: 0.0068 - val_loss: 0.0059 - val_mean_squared_error: 0.0059
Epoch 26/200
200484/200484 [==============================] - 2s 12us/step - loss: 0.0068 - mean_squared_error: 0.0068 - val_loss: 0.0054 - val_mean_squared_e

Epoch 69/200
200484/200484 [==============================] - 3s 15us/step - loss: 0.0068 - mean_squared_error: 0.0068 - val_loss: 0.0058 - val_mean_squared_error: 0.0058
Epoch 70/200
200484/200484 [==============================] - 3s 14us/step - loss: 0.0068 - mean_squared_error: 0.0068 - val_loss: 0.0054 - val_mean_squared_error: 0.0053
Epoch 71/200
200484/200484 [==============================] - 3s 14us/step - loss: 0.0068 - mean_squared_error: 0.0068 - val_loss: 0.0057 - val_mean_squared_error: 0.0057
Epoch 72/200
200484/200484 [==============================] - 3s 13us/step - loss: 0.0068 - mean_squared_error: 0.0068 - val_loss: 0.0056 - val_mean_squared_error: 0.0055
Epoch 73/200
200484/200484 [==============================] - 3s 14us/step - loss: 0.0068 - mean_squared_error: 0.0068 - val_loss: 0.0052 - val_mean_squared_error: 0.0052
Epoch 74/200
200484/200484 [==============================] - 3s 13us/step - loss: 0.0068 - mean_squared_error: 0.0068 - val_loss: 0.0053 - val_m

Epoch 117/200
200484/200484 [==============================] - 3s 13us/step - loss: 0.0068 - mean_squared_error: 0.0067 - val_loss: 0.0059 - val_mean_squared_error: 0.0059
Epoch 118/200
200484/200484 [==============================] - 3s 13us/step - loss: 0.0068 - mean_squared_error: 0.0067 - val_loss: 0.0055 - val_mean_squared_error: 0.0055
Epoch 119/200
200484/200484 [==============================] - 3s 13us/step - loss: 0.0068 - mean_squared_error: 0.0067 - val_loss: 0.0055 - val_mean_squared_error: 0.0054
Epoch 120/200
200484/200484 [==============================] - 3s 13us/step - loss: 0.0068 - mean_squared_error: 0.0067 - val_loss: 0.0059 - val_mean_squared_error: 0.0059
Epoch 121/200
200484/200484 [==============================] - 3s 13us/step - loss: 0.0068 - mean_squared_error: 0.0067 - val_loss: 0.0057 - val_mean_squared_error: 0.0057
Epoch 122/200
200484/200484 [==============================] - 3s 13us/step - loss: 0.0068 - mean_squared_error: 0.0067 - val_loss: 0.0052 -

## Summarize tuning results
This section can be run independent of the above one, if the directory paths are given properly.

In [13]:
value_set = [1e-3, 1e-4, 1e-5, 1e-6]
num_run = NUM_REPEATS

In [14]:
df_summary = pd.DataFrame(
    index=range(num_run), 
    columns=[f'para={v:.0e}' for v in value_set])

In [15]:
for value in value_set:
    for run in range(num_run):
        for i in range(end_period + 1):
            tmp = []
            df_history = pd.read_csv(f'{sub_res}para={value:.0e}/run{run}/history/history{i}.csv', index_col=0)
            tmp.append(df_history['val_mean_squared_error'].min())
        
        df_summary.loc[run, f'para={value:.0e}'] = sum(tmp) / len(tmp)
df_summary = df_summary.astype('float32')

In [16]:
df_summary.describe()

,para=1e-03,para=1e-04,para=1e-05,para=1e-06
count,5.000000,5.000000,5.000000,5.000000
mean,0.005032,0.005004,0.005052,0.005027
std,0.000042,0.000090,0.000064,0.000024
min,0.004995,0.004862,0.004949,0.004996
25%,0.005004,0.004972,0.005044,0.005016
50%,0.005009,0.005050,0.005058,0.005027
75%,0.005058,0.005052,0.005094,0.005036
max,0.005093,0.005086,0.005116,0.005060


In [17]:
df_summary.describe().to_csv(f'{sub_res}summary_metrics.csv')

In [18]:
with open(f'{sub_res}tuning-setting.txt', 'w') as f:
    f.write(f'Date and time: {datetime.datetime.now()},\n')
    f.write(f'Hypers: {hypers},\n')
    f.write(f'{lab_tune} is being tuned.\n')
    f.write(f'Value set: {value_set},\n')
    f.write(f'End period: {end_period},\n')
    f.write(f'Number of repeats: {num_run}\n')
    f.write(f'{lab_tune} is being tuned!')